# 🍓 Strawberry Dataset Overview

This notebook provides a comprehensive overview of the **LAST-Straw** (Large-Scale Annotated Synthetic Strawberry) dataset.

### Dataset Features:
1. **BBox Visualization** - Drawing bounding boxes from COCO annotations
2. **Mask Visualization** - Rendering instance masks from segmentation colors
3. **Depth Analysis** - Loading 16-bit depth maps and camera intrinsics
4. **Matching** - Understanding parent-child relationships (strawberry-peduncle)

## 1. Setup and Environment

We clone the repository to get the latest scripts and dataset (if included).

In [ ]:
!git clone https://github.com/NeverGonnaGiveYouUp/LAST-Straw-dataset.git
%cd LAST-Straw-dataset

# Install dependencies if needed (most are pre-installed on Kaggle)
!pip install -q ultralytics

In [ ]:
import json
import numpy as np
from pathlib import Path
from PIL import Image
import cv2
import matplotlib.pyplot as plt
from typing import Dict, List, Tuple, Optional

# Adjust this path based on where your dataset is located
DATASET_PATH = Path("strawberry_dataset")

if not DATASET_PATH.exists():
    # Check if it's in a Kaggle input folder instead
    DATASET_PATH = Path("/kaggle/input/last-strawberry-dataset/strawberry_dataset")

print(f"Dataset path: {DATASET_PATH.absolute()}")
print(f"Dataset exists: {DATASET_PATH.exists()}")

CATEGORIES = {
    0: {"name": "strawberry_ripe", "color": (0, 255, 0)},      # Green
    1: {"name": "strawberry_unripe", "color": (255, 0, 0)},    # Red
    2: {"name": "strawberry_half_ripe", "color": (255, 165, 0)}, # Orange
    3: {"name": "peduncle", "color": (139, 69, 19)}            # Brown
}

## 2. Helper Functions

The following functions are adapted from `Scripts/dataset_overview.py`.

In [ ]:
def load_annotations(dataset_path: Path) -> Tuple[Dict, List, List]:
    with open(dataset_path / "annotations.json", 'r') as f:
        data = json.load(f)
    return data, data.get("images", []), data.get("annotations", [])

def get_annotations_for_image(annotations: List, image_id: int) -> List:
    return [ann for ann in annotations if ann["image_id"] == image_id]

def draw_bboxes(image: np.ndarray, annotations: List, categories: Dict = CATEGORIES) -> np.ndarray:
    if image.shape[-1] == 4: image = image[:, :, :3]
    result = image.copy()
    for ann in annotations:
        bbox = ann["bbox"]
        x, y, w, h = int(bbox[0]), int(bbox[1]), int(bbox[2]), int(bbox[3])
        cat_id = ann["category_id"]
        color = categories.get(cat_id, {"color": (128, 128, 128)})["color"]
        cv2.rectangle(result, (x, y), (x + w, y + h), color, 2)
        label = f"{categories.get(cat_id, {'name': 'unk'})['name'][:4]}_{ann['instance_id']}"
        cv2.putText(result, label, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)
    return result

def load_depth_png(depth_path: Path) -> np.ndarray:
    img = Image.open(depth_path)
    depth_arr = np.array(img)
    if len(depth_arr.shape) == 3 and depth_arr.shape[2] >= 2:
        high = depth_arr[:, :, 0].astype(np.uint16)
        low = depth_arr[:, :, 1].astype(np.uint16)
        depth_mm = (high << 8) | low
        return depth_mm.astype(np.float32) / 1000.0
    return depth_arr.astype(np.float32) / 1000.0

def visualize_matching(image: np.ndarray, annotations: List, image_id: int) -> np.ndarray:
    if image.shape[-1] == 4: image = image[:, :, :3]
    result = image.copy()
    img_anns = [a for a in annotations if a["image_id"] == image_id]
    strawberries = [a for a in img_anns if a["category_id"] in [0, 1, 2]]
    peduncles = {p["instance_id"]: p for p in img_anns if p["category_id"] == 3}
    
    for straw in strawberries:
        parent_id = straw["parent_id"]
        if parent_id in peduncles:
            p1 = (int(straw["bbox"][0] + straw["bbox"][2]/2), int(straw["bbox"][1] + straw["bbox"][3]/2))
            ped = peduncles[parent_id]
            p2 = (int(ped["bbox"][0] + ped["bbox"][2]/2), int(ped["bbox"][1] + ped["bbox"][3]/2))
            cv2.line(result, p1, p2, (255, 255, 255), 2)
            cv2.circle(result, p1, 5, CATEGORIES[straw["category_id"]]["color"], -1)
            cv2.circle(result, p2, 5, CATEGORIES[3]["color"], -1)
    return result

## 3. Dataset Exploration

In [ ]:
data, images, annotations = load_annotations(DATASET_PATH)
print(f"Loaded {len(images)} images and {len(annotations)} annotations")

# Show sample visualizations for first image
img_id = 0
img_info = images[img_id]
img_path = DATASET_PATH / "images" / img_info["file_name"]
depth_path = DATASET_PATH / "depth" / img_info["file_name"]

rgb = np.array(Image.open(img_path).convert("RGB"))
depth = load_depth_png(depth_path)
img_anns = get_annotations_for_image(annotations, img_id)

fig, axes = plt.subplots(1, 3, figsize=(20, 10))

# BBoxes
axes[0].imshow(draw_bboxes(rgb, img_anns))
axes[0].set_title("Bounding Boxes")
axes[0].axis('off')

# Depth
axes[1].imshow(depth, cmap='turbo')
axes[1].set_title("Depth Map")
axes[1].axis('off')

# Matching
axes[2].imshow(visualize_matching(rgb, annotations, img_id))
axes[2].set_title("Strawberry-Peduncle Matching")
axes[2].axis('off')

plt.tight_layout()
plt.show()